In [2]:
import pandas as pd
import numpy as np
import os

path = 'D:\\maran10\\Documents\\日常工作\\6.数据复盘\\'
path_goldeye = path+'1.2020H1黄金眼\\原数据\\'
path_goldeye_new = path+'1.2020H1黄金眼\\加上行业后\\'
# path_goods = path+'全sku商品池\\c_商用订单明细_（2019H1和2020H1）\\年份品类拆分\\'

In [11]:
category = '电脑数码'
third_class = pd.read_excel(path+'1.2020H1黄金眼\\{}-H1数据展示.xlsx'.format(category), sheet_name='黄金眼top三级分类', header=1)
third_class_names = list(third_class['三级分类名称'].dropna())
third_class_names

['笔记本',
 '台式机',
 '硒鼓',
 '纸类',
 '打印机',
 '服务器/工作站',
 '办公文具',
 '路由器',
 '显示器',
 '平板电脑',
 '线缆',
 '音箱/音响',
 '文件管理',
 '投影机',
 '笔类',
 '会议音频视频',
 '耳机/耳麦',
 '工业安防监控',
 '显卡',
 'SSD固态硬盘']

In [16]:
# 读取数据 黄金眼
lst = os.listdir(path_goldeye_new+'{}\\'.format(category))
gold_df = pd.DataFrame()
gold_dict = {}
for i, p in enumerate(lst):
    if '.xlsx' not in p:
        continue
    print('reading excel:', path_goldeye_new+'{}\\{}'.format(category,p))
    gold_dict[i] = pd.read_excel(path_goldeye_new+'{}\\{}'.format(category,p),usecols=['三级分类','品牌','订单金额','企业名称'])
gold_df = pd.concat([gold_dict[i] for i in gold_dict.keys()])
print(gold_df.shape)
gold_df

(4744136, 4)


,企业名称,三级分类,品牌,订单金额
0,三棵树涂料股份有限公司,笔记本,ThinkPad,4499.0
1,三棵树涂料股份有限公司,笔类,晨光（M&G）,695.0
2,三棵树涂料股份有限公司,本册/便签,得力（deli）,46.0
3,三棵树涂料股份有限公司,电池/充电器,南孚,717.0
4,三棵树涂料股份有限公司,笔类,晨光（M&G）,418.0
...,...,...,...,...
460540,中国电信集团有限公司,监控摄像,乐橙（LeChange）,-28.0
460541,四川交投中油能源有限公司,显示器,联想（Lenovo）,-43.0
460542,中国电信集团有限公司,本册/便签,西玛（SIMAA）,-45.0
460543,中国电信集团公司山东省电信分公司,监控摄像,海康威视（HIKVISION）,-80.0


In [20]:
# 读取数据 sku
sku_df = pd.read_excel(path+"2.全sku商品池\\c_商用订单明细_（2019H1和2020H1）\\年份品类拆分\\2020H1\\('{}事业部', '2020')H1商用商品池订单明细.xlsx".format(category),
                          usecols=['三级分类','品牌','gmv','企业名称'])
print(sku_df.shape)
sku_df

(586660, 4)


,三级分类,品牌,gmv,企业名称
0,会议音频视频,maxhub,0.0,创生医疗器械（中国）有限公司
1,会议音频视频,maxhub,0.0,创生医疗器械（中国）有限公司
2,会议音频视频,maxhub,0.0,创生医疗器械（中国）有限公司
3,会议音频视频,皓丽,4363.0,台州鸿润物流包装技术有限公司
4,打印机,佳能,1988.0,中国铁路北京局集团有限公司
...,...,...,...,...
586655,路由器,普联,267.0,宿迁市优思文化艺术培训中心
586656,打印机,惠普,1899.0,江苏省财政厅
586657,投影配件,得力,310.0,江苏省财政厅
586658,投影配件,得力,1164.0,中国安能建设集团有限公司


In [34]:
gold_df_res = pd.DataFrame()
sku_df_res = pd.DataFrame()
for c in third_class_names:
    gold_df_c = gold_df[gold_df['三级分类'] == c]
    amount = gold_df_c.groupby('品牌')[['订单金额']].sum()
    client = gold_df_c.groupby('品牌')[['企业名称']].nunique()
    amount_client = pd.merge(amount, client, how='inner', left_index=True, right_index=True)
    amount_client.sort_values('订单金额', ascending=False, inplace=True)
    amount_client = amount_client.iloc[:10]
    gold_df_c_res = pd.DataFrame({'三级分类名称': [c] * len(amount_client),
                                 '品牌': list(amount_client.index),
                                 '销售额': list(amount_client['订单金额']),
                                 '客户数量': list(amount_client['企业名称'])})
    gold_df_res = pd.concat([gold_df_res, gold_df_c_res])
    
    sku_df_c = sku_df[sku_df['三级分类'] == c]
    amount2 = sku_df_c.groupby('品牌')[['gmv']].sum()
    client2 = sku_df_c.groupby('品牌')[['企业名称']].nunique()
    amount_client2 = pd.merge(amount2, client2, how='inner', left_index=True, right_index=True)
    amount_client2.sort_values('gmv', ascending=False, inplace=True)
    amount_client2 = amount_client2.iloc[:10]
    sku_df_c_res = pd.DataFrame({'三级分类名称': [c] * len(amount_client2),
                                 '品牌': list(amount_client2.index),
                                 '销售额': list(amount_client2['gmv']),
                                 '客户数量': list(amount_client2['企业名称'])})
    sku_df_res = pd.concat([sku_df_res, sku_df_c_res])
    
# gold_df_res['三级分类名称_品牌'] = gold_df_res['三级分类名称'] + gold_df_res['品牌']
# sku_df_res['三级分类名称_品牌'] = sku_df_res['三级分类名称'] + sku_df_res['品牌']
    
# df_res = pd.merge(gold_df_res, sku_df_res[['三级分类名称_品牌','销售额（全sku）','客户数量（全sku）']], how='left',
#                   left_on='三级分类名称_品牌', right_on='三级分类名称_品牌')
# del df_res['三级分类名称_品牌']
# df_res

print(gold_df_res)
print(sku_df_res)

     三级分类名称           品牌           销售额  客户数量
0       笔记本     ThinkPad  1.309514e+08   749
1       笔记本   联想（Lenovo）  7.546825e+07   574
2       笔记本        Apple  5.335445e+07     5
3       笔记本   华为（HUAWEI）  4.618555e+07   533
4       笔记本     戴尔（DELL）  2.416553e+07   384
..      ...          ...           ...   ...
5   SSD固态硬盘   联想（Lenovo）  1.328338e+06   307
6   SSD固态硬盘  闪迪（SanDisk）  1.132819e+06   234
7   SSD固态硬盘  东芝（TOSHIBA）  7.548499e+05   186
8   SSD固态硬盘  希捷（SEAGATE）  4.288443e+05   119
9   SSD固态硬盘         acer  3.880993e+05   138

[200 rows x 4 columns]
    三级分类名称        品牌           销售额    客户数量
0      笔记本  ThinkPad  5.564420e+08  8638.0
1      笔记本        华为  1.241283e+08  4344.0
2      笔记本        扬天  7.624872e+07  2248.0
3      笔记本        惠普  4.312908e+07  1794.0
4      笔记本        联想  3.639685e+07   901.0
..     ...       ...           ...     ...
5   工业安防监控       鲁凤来  7.005181e+04    11.0
6   工业安防监控       AOC  6.156000e+04     1.0
7   工业安防监控     dahua  5.557263e+04    18.0
8   工业